In [2]:
import digitalhub as dh
import pandas as pd
import requests
import os

In [3]:
PROJECT = "project-etl-ci"
project = dh.get_or_create_project(PROJECT)

In [4]:
print(project)

{'kind': 'project', 'metadata': {'project': 'project-etl-ci', 'name': 'project-etl-ci', 'created': '2024-10-24T11:20:17.542Z', 'updated': '2024-10-24T11:20:17.542Z', 'created_by': 'khurshid@fbk.eu', 'updated_by': 'khurshid@fbk.eu'}, 'spec': {'context': './', 'functions': [], 'artifacts': [], 'workflows': [], 'dataitems': [], 'models': []}, 'status': {'state': 'CREATED'}, 'user': 'khurshid@fbk.eu', 'id': 'project-etl-ci', 'name': 'project-etl-ci', 'key': 'store://project-etl-ci'}


In [5]:
URL = "https://opendata.comune.bologna.it/api/explore/v2.1/catalog/datasets/rilevazione-flusso-veicoli-tramite-spire-anno-2023/exports/csv?lang=it&timezone=Europe%2FRome&use_labels=true&delimiter=%3B"
filename = "rilevazione-flusso-veicoli-tramite-spire-anno-2023.csv"

In [6]:
with requests.get(URL) as r:
    with open(filename, "wb") as f:
        f.write(r.content)

In [9]:
df = pd.read_csv(filename, sep=";")

In [10]:
df

,data,codice spira,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,06:00-07:00,07:00-08:00,...,ordinanza,stato,codimpsem,direzione,angolo,longitudine,latitudine,geopoint,ID_univoco_stazione_spira,giorno settimana
0,2023-04-01,0.127 1.5 2 1,34,31,14,7,9,12,22,48,...,4000/343434,A,250,S,163.0,11.343595,44.501688,"44.5016875497437, 11.3435953508993",16.0,Sabato
1,2023-04-01,0.127 1.5 5 1,38,29,20,6,13,12,18,21,...,4000/343434,A,250,N,343.0,11.343408,44.501173,"44.5011734938257, 11.3434083495971",17.0,Sabato
2,2023-04-01,0.127 1.13 6 1,0,0,0,0,0,0,0,0,...,4000/343434,A,323,S,198.0,11.346420,44.491648,"44.4916483847646, 11.3464200565732",3.0,Sabato
3,2023-04-01,0.127 1.15 6 1,68,50,38,13,12,18,32,43,...,4000/343434,A,324,S,168.0,11.343443,44.490027,"44.4900271601578, 11.3434427397635",7.0,Sabato
4,2023-04-01,0.127 1.82 2 2,86,55,37,30,27,23,51,119,...,4000/343434,A,85,SE,217.0,11.349800,44.490458,"44.4904584072207, 11.34979980231",20.0,Sabato
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313062,2023-12-31,4.91 4.90 6 1,22,20,7,7,4,5,10,16,...,4000/343434,A,192,SE,212.0,11.382992,44.476845,"44.4768446377012, 11.3829922132726",904.0,Domenica
313063,2023-12-31,4.95 4.94 6 1,59,31,15,9,5,13,18,34,...,4000/343434,A,187,E,242.0,11.362150,44.486843,"44.4868432793297, 11.3621500984113",918.0,Domenica
313064,2023-12-31,4.98 0.127 4 1,43,20,7,7,6,13,15,15,...,4000/343434,A,176,NO,69.0,11.385065,44.469348,"44.4693478190501, 11.3850649221279",920.0,Domenica
313065,2023-12-31,4.98 0.127 8 1,4,4,2,1,1,0,2,0,...,4000/343434,A,176,E,257.0,11.385614,44.469066,"44.469065830958, 11.3856143249703",922.0,Domenica


In [11]:
new_folder = 'src'
if not os.path.exists(new_folder):
    os.makedirs(new_folder)

In [12]:
%%writefile "src/download-data.py"

from digitalhub_runtime_python import handler

@handler(outputs=["dataset"])
def downloader(url):
    # read and rewrite to normalize and export as data
    df = url.as_df(file_format='csv',sep=";")
    return df

Writing src/download-data.py


In [13]:
func = project.new_function(
                         name="download-data",
                         kind="python",
                         python_version="PYTHON3_10",
                         code_src="src/download-data.py",
                         handler="downloader")

In [14]:
URL = "https://opendata.comune.bologna.it/api/explore/v2.1/catalog/datasets/rilevazione-flusso-veicoli-tramite-spire-anno-2023/exports/csv?lang=it&timezone=Europe%2FRome&use_labels=true&delimiter=%3B"
di= project.new_dataitem(name="url_data_item",kind="table",path=URL)

In [15]:
run = func.run(action="job", inputs={'url':di.key}, outputs={"dataset": "dataset"}, local_execution=False)

In [16]:
dataset_di = project.get_dataitem('dataset')

In [17]:
dataset_df = dataset_di.as_df()

In [18]:
dataset_df

,data,codice spira,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,06:00-07:00,07:00-08:00,...,ordinanza,stato,codimpsem,direzione,angolo,longitudine,latitudine,geopoint,ID_univoco_stazione_spira,giorno settimana
0,2023-04-01,0.127 1.5 2 1,34,31,14,7,9,12,22,48,...,4000/343434,A,250,S,163.0,11.343595,44.501688,"44.5016875497437, 11.3435953508993",16.0,Sabato
1,2023-04-01,0.127 1.5 5 1,38,29,20,6,13,12,18,21,...,4000/343434,A,250,N,343.0,11.343408,44.501173,"44.5011734938257, 11.3434083495971",17.0,Sabato
2,2023-04-01,0.127 1.13 6 1,0,0,0,0,0,0,0,0,...,4000/343434,A,323,S,198.0,11.346420,44.491648,"44.4916483847646, 11.3464200565732",3.0,Sabato
3,2023-04-01,0.127 1.15 6 1,68,50,38,13,12,18,32,43,...,4000/343434,A,324,S,168.0,11.343443,44.490027,"44.4900271601578, 11.3434427397635",7.0,Sabato
4,2023-04-01,0.127 1.82 2 2,86,55,37,30,27,23,51,119,...,4000/343434,A,85,SE,217.0,11.349800,44.490458,"44.4904584072207, 11.34979980231",20.0,Sabato
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313062,2023-12-31,4.91 4.90 6 1,22,20,7,7,4,5,10,16,...,4000/343434,A,192,SE,212.0,11.382992,44.476845,"44.4768446377012, 11.3829922132726",904.0,Domenica
313063,2023-12-31,4.95 4.94 6 1,59,31,15,9,5,13,18,34,...,4000/343434,A,187,E,242.0,11.362150,44.486843,"44.4868432793297, 11.3621500984113",918.0,Domenica
313064,2023-12-31,4.98 0.127 4 1,43,20,7,7,6,13,15,15,...,4000/343434,A,176,NO,69.0,11.385065,44.469348,"44.4693478190501, 11.3850649221279",920.0,Domenica
313065,2023-12-31,4.98 0.127 8 1,4,4,2,1,1,0,2,0,...,4000/343434,A,176,E,257.0,11.385614,44.469066,"44.469065830958, 11.3856143249703",922.0,Domenica


In [19]:
%%writefile "src/process-spire.py"

from digitalhub_runtime_python import handler

KEYS=['codice spira','longitudine','latitudine','Livello','tipologia','codice','codice arco','codice via','Nome via', 'stato','direzione','angolo','geopoint']

@handler(outputs=["dataset-spire"])
def process(project, di):
    df = di.as_df()
    sdf= df.groupby(['codice spira']).first().reset_index()[KEYS]
    return sdf

Writing src/process-spire.py


In [20]:
process_func = project.new_function(
                         name="process-spire",
                         kind="python",
                         python_version="PYTHON3_10",
                         code_src="src/process-spire.py",
                         handler="process")

In [21]:
process_run = process_func.run(action="job", inputs={'di': dataset_di.key}, outputs={'dataset-spire': 'dataset-spire'}, local_execution=False)

In [22]:
spire_di = project.get_dataitem('dataset-spire')
spire_df = spire_di.as_df()

In [23]:
spire_df

,codice spira,longitudine,latitudine,Livello,tipologia,codice,codice arco,codice via,Nome via,stato,direzione,angolo,geopoint
0,0.127 1.1 6 1,11.354166,44.498535,1,spira,498.0,3312,19900,VIA G.BATTISTA DE ROLANDIS,A,N,342.0,"44.4985349106485, 11.3541657967424"
1,0.127 1.12 8 1,11.338970,44.495251,1,spira,1045.0,1016,5900,VIA CESARE BATTISTI,A,N,350.0,"44.4952505129043, 11.338970003537"
2,0.127 1.13 6 1,11.346420,44.491648,1,spira,130.0,1169,14700,VIA CASTIGLIONE,A,S,198.0,"44.4916483847646, 11.3464200565732"
3,0.127 1.14 4 1,11.339836,44.490116,1,spira,521.0,1050,59900,VIA URBANA,A,E,264.0,"44.4901162203284, 11.3398356513878"
4,0.127 1.15 2 1,11.343358,44.489507,1,spira,132.0,1064,25800,VIA GARIBALDI,A,N,347.0,"44.4895074220971, 11.3433581064329"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
900,4.98 0.127 4 1,11.385065,44.469348,1,spira,946.0,1989,15350,VIA FRANCESCO CAVAZZONI,A,NO,69.0,"44.4693478190501, 11.3850649221279"
901,4.98 0.127 6 1,11.385481,44.469397,1,spira,899.0,2002,23230,VIA FIRENZE,A,N,352.0,"44.4693967343944, 11.385480982166"
902,4.98 0.127 8 1,11.385614,44.469066,1,spira,912.0,1960,42780,VIA OSOPPO,A,E,257.0,"44.469065830958, 11.3856143249703"
903,4.98 4.98 3 1,11.385260,44.468970,1,spira,1067.0,1963,32510,VIA LUIGI LONGO,A,S,170.0,"44.4689700816957, 11.3852604232066"


In [24]:
%%writefile "src/process-measures.py"

from digitalhub_runtime_python import handler
import pandas as pd

KEYS = ['00:00-01:00', '01:00-02:00', '02:00-03:00', '03:00-04:00', '04:00-05:00', '05:00-06:00', '06:00-07:00', '07:00-08:00', '08:00-09:00', '09:00-10:00', '10:00-11:00', '11:00-12:00', '12:00-13:00', '13:00-14:00', '14:00-15:00', '15:00-16:00', '16:00-17:00', '17:00-18:00', '18:00-19:00', '19:00-20:00', '20:00-21:00', '21:00-22:00', '22:00-23:00', '23:00-24:00']
COLUMNS=['data','codice spira']

@handler(outputs=["dataset-measures"])
def process(project, di):
    df = di.as_df()
    rdf = df[COLUMNS+KEYS]
    ls = []
    for key in KEYS:
        k = key.split("-")[0]
        xdf = rdf[COLUMNS + [key]]
        xdf['time'] = xdf.data.apply(lambda x: x+' ' +k)
        xdf['value'] = xdf[key]
        ls.append(xdf[['time','codice spira','value']])
    edf = pd.concat(ls)
    return edf

Writing src/process-measures.py


In [25]:
process_measures_func = project.new_function(
                         name="process-measures",
                         kind="python",
                         python_version="PYTHON3_10",
                         code_src="src/process-measures.py",
                         handler="process")

In [26]:
process_measures_run = process_measures_func.run(action="job", inputs={'di': dataset_di.key}, outputs={'dataset-measures': 'dataset-measures'}, local_execution=False)

In [27]:
measures_di = project.get_dataitem('dataset-measures')
measures_df = measures_di.as_df()
measures_df.head()

,time,codice spira,value
0,2023-04-01 00:00,0.127 1.5 2 1,34
1,2023-04-01 00:00,0.127 1.5 5 1,38
2,2023-04-01 00:00,0.127 1.13 6 1,0
3,2023-04-01 00:00,0.127 1.15 6 1,68
4,2023-04-01 00:00,0.127 1.82 2 2,86


In [28]:
%%writefile "src/pipeline.py"

from digitalhub_runtime_kfp.dsl import pipeline_context

def pipeline(url):
    with pipeline_context() as pc:
        downloader = pc.step(
            name="download-data",
            function="download-data",
            action="job",
            inputs={"url": url},
            outputs={"dataset": "dataset"},
        )

        process_spire = pc.step(
            name="process-spire",
            function="process-spire",
            action="job",
            inputs={"di": downloader.outputs["dataset"]}
        )

        process_measures = pc.step(
            name="process-measures",
            function="process-measures",
            action="job",
            inputs={"di": downloader.outputs["dataset"]}
        )

Writing src/pipeline.py


In [29]:
workflow = project.new_workflow(name="pipeline", kind="kfp", code_src="src/pipeline.py", handler="pipeline")

In [30]:
workflow.run(parameters={"url": di.key})

{'kind': 'kfp+run', 'metadata': {'project': 'project-etl-ci', 'name': 'effae55d-2d8b-4136-b751-5f9cce83db8e', 'created': '2024-10-24T11:26:11.16Z', 'updated': '2024-10-24T11:26:11.176Z', 'created_by': 'khurshid@fbk.eu', 'updated_by': 'khurshid@fbk.eu'}, 'spec': {'task': 'kfp+pipeline://project-etl-ci/pipeline:051163dd-cbe0-4c50-8ea2-84fd295918c9', 'local_execution': False, 'function': 'kfp://project-etl-ci/pipeline:051163dd-cbe0-4c50-8ea2-84fd295918c9', 'source': {'source': 'src/pipeline.py', 'handler': 'pipeline', 'base64': 'CmZyb20gZGlnaXRhbGh1Yl9ydW50aW1lX2tmcC5kc2wgaW1wb3J0IHBpcGVsaW5lX2NvbnRleHQKCmRlZiBwaXBlbGluZSh1cmwpOgogICAgd2l0aCBwaXBlbGluZV9jb250ZXh0KCkgYXMgcGM6CiAgICAgICAgZG93bmxvYWRlciA9IHBjLnN0ZXAoCiAgICAgICAgICAgIG5hbWU9ImRvd25sb2FkLWRhdGEiLAogICAgICAgICAgICBmdW5jdGlvbj0iZG93bmxvYWQtZGF0YSIsCiAgICAgICAgICAgIGFjdGlvbj0iam9iIiwKICAgICAgICAgICAgaW5wdXRzPXsidXJsIjogdXJsfSwKICAgICAgICAgICAgb3V0cHV0cz17ImRhdGFzZXQiOiAiZGF0YXNldCJ9LAogICAgICAgICkKCiAgICAgICAgcHJvY2Vzc19zcGlyZSA9

In [31]:
%%writefile 'src/api.py'

def init_context(context):
    di = context.project.get_dataitem('dataset-measures')
    df = di.as_df()
    setattr(context, "df", df)

def handler(context, event):
    df = context.df

    if df is None:
        return ""

    # mock REST api
    method = event.method
    path = event.path
    fields = event.fields

    id = False

    # pagination
    page = 0
    pageSize = 50

    if "page" in fields:
        page = int(fields['page'])

    if "size" in fields:
        pageSize = int(fields['size'])

    if page < 0:
        page = 0

    if pageSize < 1:
        pageSize = 1

    if pageSize > 100:
        pageSize = 100

    start = page * pageSize
    end = start + pageSize
    total = len(df)

    if end > total:
        end = total

    ds = df.iloc[start:end]
    json = ds.to_json(orient="records")

    res = {"data": json, "page": page, "size": pageSize, "total": total}

    return res

Writing src/api.py


In [32]:
api_func = project.new_function(
                         name="api",
                         kind="python",
                         python_version="PYTHON3_10",
                         code_src="src/api.py",
                         handler="handler",
                         init_function="init_context")

In [33]:
run_serve_model = api_func.run(action="serve")

In [36]:
run_serve_model.refresh()

{'kind': 'python+run', 'metadata': {'project': 'project-etl-ci', 'name': 'cdfebd09-280b-486d-87cc-a47ca819da70', 'created': '2024-10-24T11:26:26.128Z', 'updated': '2024-10-24T11:27:28.594Z', 'created_by': 'khurshid@fbk.eu', 'updated_by': 'khurshid@fbk.eu'}, 'spec': {'task': 'python+serve://project-etl-ci/api:02125fed-2efb-48ea-a688-f90d6d6aab21', 'local_execution': False, 'function': 'python://project-etl-ci/api:02125fed-2efb-48ea-a688-f90d6d6aab21', 'source': {'source': 'src/api.py', 'handler': 'handler', 'base64': 'CmRlZiBpbml0X2NvbnRleHQoY29udGV4dCk6CiAgICBkaSA9IGNvbnRleHQucHJvamVjdC5nZXRfZGF0YWl0ZW0oJ2RhdGFzZXQtbWVhc3VyZXMnKQogICAgZGYgPSBkaS5hc19kZigpCiAgICBzZXRhdHRyKGNvbnRleHQsICJkZiIsIGRmKQoKZGVmIGhhbmRsZXIoY29udGV4dCwgZXZlbnQpOgogICAgZGYgPSBjb250ZXh0LmRmCgogICAgaWYgZGYgaXMgTm9uZToKICAgICAgICByZXR1cm4gIiIKCiAgICAjIG1vY2sgUkVTVCBhcGkKICAgIG1ldGhvZCA9IGV2ZW50Lm1ldGhvZAogICAgcGF0aCA9IGV2ZW50LnBhdGgKICAgIGZpZWxkcyA9IGV2ZW50LmZpZWxkcwoKICAgIGlkID0gRmFsc2UKCiAgICAjIHBhZ2luYXRpb24KICAgI

In [37]:
SERVICE_URL = f"http://{run_serve_model.status.to_dict()['service']['url']}"
SERVICE_URL

'http://s-pythonserve-cdfebd09-280b-486d-87cc-a47ca819da70.digitalhub-tenant1:8080'

In [38]:
with requests.get(f'{SERVICE_URL}/?page=5&size=10') as r:
    res = r.json()
print(res)

{'data': '[{"time":"2023-04-01 00:00","codice spira":"0.127 3.42 8 1","value":91},{"time":"2023-04-01 00:00","codice spira":"0.127 3.45 4 1","value":83},{"time":"2023-04-01 00:00","codice spira":"0.127 3.46 10 1","value":50},{"time":"2023-04-01 00:00","codice spira":"0.127 3.50 2 1","value":101},{"time":"2023-04-01 00:00","codice spira":"0.127 3.64 4 1","value":5},{"time":"2023-04-01 00:00","codice spira":"0.127 3.64 6 1","value":455},{"time":"2023-04-01 00:00","codice spira":"0.127 3.69 2 1","value":571},{"time":"2023-04-01 00:00","codice spira":"0.127 3.83 4 1","value":17},{"time":"2023-04-01 00:00","codice spira":"0.127 3.84 8 1","value":0},{"time":"2023-04-01 00:00","codice spira":"0.127 3.86 3 1","value":108}]', 'page': 5, 'size': 10, 'total': 7513608}


In [39]:
rdf = pd.read_json(res['data'], orient='records')
rdf.head()

,time,codice spira,value
0,2023-04-01 00:00,0.127 3.42 8 1,91
1,2023-04-01 00:00,0.127 3.45 4 1,83
2,2023-04-01 00:00,0.127 3.46 10 1,50
3,2023-04-01 00:00,0.127 3.50 2 1,101
4,2023-04-01 00:00,0.127 3.64 4 1,5
